<a href="https://colab.research.google.com/github/aniketcomps/BERT-Topic-Modeling/blob/main/Unstrucutured_project_Wallstreetbets_recommendations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unstructured Analytics

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Scrapping

# Using pmaw to get big blocks of comments and posts

In [ ]:
!pip install pmaw
!pip install bertopic
!pip install --upgrade joblib==1.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 188 kB 7.1 MB/s 
     |████████████████████████████████| 55 kB 3.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 4.2 MB/s 
     |████████████████████████████████| 85 kB 4.4 MB/s 
     |████████████████████████████████| 636 kB 46.6 MB/s 
     |████████████████████████████████| 5.2 MB 42.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 88 kB 6.3 MB/s 
     |████████████████████████████████| 5.3 MB 42.1 MB/s 
     |████████████████████████████████| 1.3 MB 50.2 MB/s 
     |████████████████████████████████| 163 kB 39.7 MB/s 
     |████████████████████████████████| 7.6 MB 33.8 MB/s 
     |████████████████████████████████| 1.1 MB 

In [ ]:
# pip install --upgrade joblib==1.1.0 this is needed now because of conflicts with joblib and bertopic
from bertopic import BERTopic

In [ ]:
import pandas as pd
from pmaw import PushshiftAPI
api = PushshiftAPI()
import re

In [ ]:
import datetime as dt
before = int(dt.datetime(2022,10,1,0,0).timestamp())
after = int(dt.datetime(2022,9,1,0,0).timestamp())

In [ ]:
# # this takes a long time to run so avoid as much as possible
# # https://pypi.org/project/pmaw/#description
# subreddit="wallstreetbets"
# limit=500000


# comments = api.search_comments(subreddit=subreddit, limit=limit, before=before, after=after)
# submissions = api.search_submissions(subreddit=subreddit, limit=limit, before=before, after=after)

# comments_pd = pd.DataFrame(comments)
# submissions_pd = pd.DataFrame(submissions)

# comments_pd.to_csv('/content/drive/MyDrive/unstructured_project/wsb_comments.csv')
# submissions_pd.to_csv('/content/drive/MyDrive/unstructured_project/wsb_posts.csv')

In [ ]:
# use this instead - if the previous one is commented out
comments_pd = pd.read_csv('/content/drive/MyDrive/unstructured_project/wsb_comments.csv')
submissions_pd = pd.read_csv('/content/drive/MyDrive/unstructured_project/wsb_posts.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0,3,6,17,18,19,20,21,22,23,24,26,29,31,32,37,38,39,49,51) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4,83) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
commentslist = comments_pd[comments_pd['score'] > 1]['body'].tolist() #only getting comments with more than 5 upvotes
submissionslist = submissions_pd[submissions_pd['score'] > 1]['selftext'].tolist() #only getting posts with more than 5 upvotes

# combine posts and comments
combined = submissionslist + commentslist

In [ ]:
combined = [str(x) for x in combined]
combined = [x for x in combined if len(x) > 30]  # removing sentences with less words (less than 30)

In [ ]:
# https://maartengr.github.io/BERTopic/api/ctfidf.html
# https://github.com/pinecone-io/examples/blob/master/learn/algos-and-libraries/bertopic/01_topic_modeling.ipynb
from sklearn.feature_extraction.text import CountVectorizer

# we add this to remove stopwords, for lower volumes of data stopwords can cause issues
# vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words="english")

model = BERTopic(
    # vectorizer_model=vectorizer_model,
    language='english', calculate_probabilities=True,
    verbose=True
)
topics, probs = model.fit_transform(combined)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/118 [00:00<?, ?it/s]

2022-10-14 20:45:53,969 - BERTopic - Transformed documents to Embeddings
2022-10-14 20:46:24,798 - BERTopic - Reduced dimensionality
2022-10-14 20:46:25,796 - BERTopic - Clustered reduced embeddings


In [ ]:
freq = model.get_topic_info()
freq.head(30)
# outliers are under topic -1 - they need to be ignored and should be taken into analysis (generally stop words fall under this)

,Topic,Count,Name
0,-1,1394,-1_the_to_you_is
1,0,149,0_puts_bought_my_buying
2,1,124,1_spy_400_down_if
3,2,108,2_bulls_are_bull_to
4,3,102,3_inflation_edt0400_bloomberg_walter
5,4,96,4_imgemotet52th524260_imgemotet52th524271_imge...
6,5,96,5_money_trading_trade_you
7,6,79,6_390_we_380_spx
8,7,79,7_calls_bought_my_buying
9,8,70,8_eat_chicken_whiskey_diet


In [ ]:
a_topic = freq.iloc[3]['Topic'] 
model.get_topic(a_topic)

[('bulls', 0.12094768259045433),
 ('are', 0.023051255137253764),
 ('bull', 0.0176572140069673),
 ('to', 0.017624589200586432),
 ('sad', 0.013606034974238993),
 ('you', 0.013558880968444887),
 ('will', 0.013499560712920355),
 ('bullish', 0.013140532057087765),
 ('we', 0.012933881531338827),
 ('the', 0.012591639980659364)]

In [ ]:
# this one is similar to LDAvis
model.visualize_topics()

In [ ]:
# Topics are clustered nicely - we see in one cluster on the bottom left - everything related to derivatives trading - options rading in particular
# middle one talks about inflation and recession 
# top right is jerome powell, elon musk and zukerberg relatiod
# top left is lifestyle and food
# middle right part is related to 

In [ ]:
# model.visualize_distribution(probs[10], min_probability=0.05)

In [ ]:
model.visualize_barchart(top_n_topics=25)

In [ ]:
model.get_topic(7)

[('apple', 0.10952836821826598),
 ('iphone', 0.07122453603094807),
 ('watch', 0.05537860216737647),
 ('new', 0.03375415143796303),
 ('14', 0.020856000727231646),
 ('gtapple', 0.019980356696893683),
 ('robot', 0.01894044464022634),
 ('watching', 0.015300729195833132),
 ('due', 0.014989592197500333),
 ('for', 0.014467663350145613)]

In [ ]:
# This shows a negative sentiment. The most popular topic is people buying puts and that bulls are 'sad'.
# We also see people discussing the queens death, the new iphone, watch and 14. queens dead and prince charles etc.

In [ ]:
model.visualize_hierarchy(top_n_topics=30)

# Get top stocks from WSB for that month

In [ ]:
!pip install praw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import praw
reddit = praw.Reddit(
    client_id="AoIAaPX3SGae39chaJt5Dw",
    client_secret="YxvaEg5nz7bnvUMPfLXxn1TitVqRSg",
    password="Animal@12345",
    user_agent="wsb_scraper",
    username="diamond_hands_007",check_for_async=False
)
print(reddit.user.me())


diamond_hands_007


In [ ]:
reddit = praw.Reddit(
    client_id="AoIAaPX3SGae39chaJt5Dw",
    client_secret="YxvaEg5nz7bnvUMPfLXxn1TitVqRSg",
    password="Animal@12345",
    user_agent="wsb_scraper",
    username="diamond_hands_007",check_for_async=False
)
print(reddit.user.me())


diamond_hands_007


In [ ]:
# running this takes a long time so avoid
def get_stock_list():
    ticker_dict = {}
    filelist = ['/content/drive/MyDrive/unstructured_project/ticker_list/list1.csv', 
                '/content/drive/MyDrive/unstructured_project/ticker_list/list2.csv',
                '/content/drive/MyDrive/unstructured_project/ticker_list/list3.csv']
    for file in filelist:
        tl = pd.read_csv(file, skiprows=0, skip_blank_lines=True)
        tl = tl[tl.columns[0]].tolist()
        for ticker in tl:
            ticker_dict[ticker] = 1
    return ticker_dict

def get_tickers(sub, stockList):
    monthlyTickers = {}

    regexPattern = r'\b([A-Z]+)\b'
    tickerDict = stockList
    blacklist = ["A", "I", "DD", "WSB", "YOLO", "RH", "EV", "PE", "ETH", "BTC", "E"] 
    for submission in reddit.subreddit(sub).top("month"):
        strings = [submission.title]
        submission.comments.replace_more(limit=0)
        for comment in submission.comments.list():
            strings.append(comment.body)
        for s in strings:
            for phrase in re.findall(regexPattern, s):
                if phrase not in blacklist:
                    if phrase in tickerDict:
                        if phrase not in monthlyTickers:
                            monthlyTickers[phrase] = 1
                        else:
                            monthlyTickers[phrase] += 1
    return monthlyTickers

stockList = get_stock_list()
topTickers = {}
for sub in ['wallstreetbets']:
    monthlyTickers = get_tickers(sub, stockList)
    for ticker in monthlyTickers.keys():
        if ticker in topTickers:
            topTickers[ticker] += monthlyTickers[ticker]
        else:
            topTickers[ticker] = monthlyTickers[ticker]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Positional arguments for 'BaseListingMixin.top' will no longer be supported in PRAW 8.
Call this function with 'time_filter' as a keyword argument.


In [ ]:
ticker_freq = pd.DataFrame(sorted(topTickers.items(), key=lambda x: x[1], reverse=True))
ticker_freq.columns = ['ticker', 'tf']
# ticker_freq.to_csv('/content/drive/MyDrive/unstructured_project/top_tickers_30days.csv')

In [ ]:
ticker_freq = pd.read_csv('/content/drive/MyDrive/unstructured_project/top_tickers_30days.csv', index_col=0)

In [ ]:
ticker_freq = ticker_freq[ticker_freq['ticker'].apply(lambda x:len(x) > 1)] #filter out single character tickers

In [ ]:
# manually removing some unwanted words that will not be tickers
set_tickers = set(ticker_freq[:50]['ticker'].tolist()).difference(set(['FOR', ' U', 'ALL', 'CAN', 'EDIT', 'GDP', 'CEO', 'CFO', 'NYC', 'ARE', 'FSD', 'NOW', 'IMO', 'USA', 'USB', 'QQQ', 'ONE',
                                                         'MSM', 'JOE', 'IRS', 'ANY', 'ATH', 'BOOM', 'UK']))

In [ ]:
ticker_freq = ticker_freq[ticker_freq['ticker'].isin(list(set_tickers))]

# Stock recommender

In [ ]:
import praw
import pandas as pd
import os
import datetime
import requests
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
wsb = reddit.subreddit('wallstreetbets')

In [ ]:
# common WSB terminology and associated sentiments, more WSB slang can be added here
# these lexicons are assigned based on the topic modelling we did - essentially its a bear market so the negative sentiment is prevalent
wsb_words = {
    'tendies': 2.0,
    'bull': 2.0,
    'bear': -10.0,
    'short': -10.0,
    'sell': -10.0,
    'puts': -10.0,
    'calls': 2.0,
    'moon': 2.0,
    'btfd': 2.0,
    'going tits up': -10.0,
}

In [ ]:
def get_sentiment(df_col, words=wsb_words):

    # initializes VADER
    vader = SentimentIntensityAnalyzer()

    # updates VADER's lexicon to include WSB terminology
    vader.lexicon.update(wsb_words)

    # analyzes sentiment of dataframe column input
    scores = df_col.apply(vader.polarity_scores).tolist()
    return pd.DataFrame(scores)

def get_date(submission):
    
    # gets submission datetime
    try:
        time = submission.created
    except: 
        time = submission
    return datetime.date.fromtimestamp(time)

def scrape_for(symbol, limit=10, before='', after='30d', subreddit='wallstreetbets'): #considered 30 days as this will take data from 1st september to date, if we want to look at a wider timeframe, change here
    result = []
    url_for_comment = 'https://api.pushshift.io/reddit/search/comment/?q={0}&subreddit={1}&after={2}&size={3}&before={4}&sort_type=score'.format(symbol, subreddit, after, limit, before)
    comments = requests.get(url_for_comment).json()['data']

    url_for_submission = 'https://api.pushshift.io/reddit/search/submission/?q={0}&subreddit={1}&after={2}&size={3}&before={4}&sort_type=score'.format(symbol, subreddit, after, limit, before)
    submissions = requests.get(url_for_submission).json()['data']

    for comment in comments:
        result.append((comment['body'], get_date(comment['created_utc'])))

    for submission in submissions:
        date = get_date(submission['created_utc'])
        try:
            result.append((submission['title'] + submission['selftext'], date))
        except:
            result.append((submission['title'], date))
        sub = reddit.submission(id=submission['id'])
        sub.comments.replace_more(limit=0)
        for top_level_comment in sub.comments:
            result.append((top_level_comment.body, date))

    # creates a dataframe from API results
    df = pd.DataFrame(result, columns=['Text', 'Date'])

    # updates dataframe to include sentiments
    scored_df = df.join(get_sentiment(df['Text']), rsuffix='_right')

    return scored_df

In [ ]:
# this takes a long time so dont run for more than 5
top_n = 5 #get compund sentiment for top n stocks from WSB using vader
ticker_freq['compound_sentiment'] = 0
for ticker in ticker_freq[:top_n]['ticker'].tolist():
  sentiment_total = scrape_for(ticker)
  ticker_freq['compound_sentiment'][ticker_freq['ticker'] == ticker] = sentiment_total['compound'].mean()

In [ ]:
# sentiment for top n stocks (over last 30 days)
ticker_freq[:top_n]

,ticker,tf,compound_sentiment
0,GME,382,0.029451
2,AI,127,-0.022523
3,BBBY,101,-0.026281
5,AMC,90,-0.049026
6,CS,85,-0.111998


In [ ]:
# dollar has a positive sentiment (obviously)
sentiment_total = scrape_for('USD')
sentiment_total['compound'].mean()

-0.04046829268292681

In [ ]:
# pound has a negative sentiment (obviously)
sentiment_total = scrape_for('sterling')
sentiment_total['compound'].mean()

-0.136907

# stock simulator - analysis in restrospect

In [ ]:
# this takes a really long time so dont run for more than 5
top_n = 10
ticker_freq['compound_sentiment'] = 0
ticker_freq['compound_sentiment_1month_back'] = 0
ticker_freq['compound_sentiment_2months_back'] = 0
for ticker in ticker_freq[:top_n]['ticker'].tolist():
  sentiment_total = scrape_for(ticker, after='90d', before='60d') #3 months back with 1 month data
  ticker_freq['compound_sentiment_2months_back'][ticker_freq['ticker'] == ticker] = sentiment_total['compound'].mean()
  sentiment_total = scrape_for(ticker, after='60d', before='30d') #2 months back with 1 month data
  ticker_freq['compound_sentiment_1month_back'][ticker_freq['ticker'] == ticker] = sentiment_total['compound'].mean()
  sentiment_total = scrape_for(ticker) #1 months back with 1 month data
  ticker_freq['compound_sentiment'][ticker_freq['ticker'] == ticker] = sentiment_total['compound'].mean()

In [ ]:
ticker_freq[:top_n]

,ticker,tf,compound_sentiment,compound_sentiment_1month_back,compound_sentiment_2months_back
0,GME,382,0.029451,-0.034960,-0.035763
2,AI,127,-0.022523,0.094232,0.144990
3,BBBY,101,-0.026281,0.007149,-0.000798
5,AMC,90,-0.049026,0.004049,0.056591
6,CS,85,-0.111998,0.149132,0.096831
9,TSLA,63,-0.011124,-0.082407,-0.052972
14,IT,44,-0.094143,-0.041592,-0.036135
18,NVDA,37,-0.062092,-0.078903,0.043307
19,GM,36,0.045823,0.184702,-0.107337
23,AMD,34,-0.007595,-0.032588,-0.009890


In [ ]:
# checking with actual stock prices
# https://github.com/atreadw1492/yahoo_fin

In [ ]:
# !pip install yahoo_fin

In [ ]:
from yahoo_fin.stock_info import get_data, tickers_sp500, tickers_nasdaq, tickers_other, get_quote_table

In [ ]:
for ticker in ticker_freq[:top_n]['ticker'].tolist():
  print(get_data(ticker, start_date = '07/01/2022' , end_date = '10/01/2022' , interval = '1mo')[['ticker', 'close']])

           ticker      close
2022-07-01    GME  34.009998
2022-08-01    GME  28.639999
2022-09-01    GME  25.129999
           ticker  close
2022-07-01     AI  18.41
2022-08-01     AI  18.00
2022-09-01     AI  12.50
           ticker  close
2022-07-01   BBBY   5.03
2022-08-01   BBBY   9.53
2022-09-01   BBBY   6.09
           ticker  close
2022-07-01    AMC  14.56
2022-08-01    AMC   9.12
2022-09-01    AMC   6.97
           ticker  close
2022-07-01     CS   5.80
2022-08-01     CS   5.16
2022-09-01     CS   3.92
           ticker       close
2022-07-01   TSLA  297.149994
2022-08-01   TSLA  275.609985
2022-09-01   TSLA  265.250000
           ticker       close
2022-07-01     IT  265.480011
2022-08-01     IT  285.320007
2022-09-01     IT  276.690002
           ticker       close
2022-07-01   NVDA  181.630005
2022-08-01   NVDA  150.940002
2022-09-01   NVDA  121.389999
           ticker      close
2022-07-01     GM  36.259998
2022-08-01     GM  38.209999
2022-09-01     GM  32.090000
        

In [ ]:
# We would have lost money only on AI and credit suisse (CS)